# IBM Advanced Data Science Capstone Project
## Sentiment Analysis of Amazon Customer Reviews
### Harsh V Singh, Apr 2021

## Model Definition

In this notebook, we will define the machine learning model that will be used to train and predict the sentiment of an Amazon customer's review given its review heading and text. We have already preprocessed the raw data into a training set containing tokenized and vectorized features of the review text content along with a binary review sentiment which is 1 for positive and 0 for negative reviews.

## Importing required Python libraries and initializing Apache Spark environment

In [ ]:
import numpy as np
import pandas as pd
import math
import time
from pathlib import Path
from scipy import sparse
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import seaborn as sns

import sklearn
from sklearn.naive_bayes import ComplementNB

from sklearn import metrics
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM, Masking, Embedding
from keras import regularizers
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.autograph.set_verbosity(0)

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, StringType, ArrayType
from pyspark.sql.functions import udf, rand, col, concat, coalesce
from pyspark.ml.feature import HashingTF, IDF

CPU_CORES = 4
conf = SparkConf().setMaster("local[*]") \
    .setAll([("spark.driver.memory", "24g"),\
             ("spark.executor.memory", "8g"), \
             ("spark.driver.maxResultSize", "24g"), \
             ("spark.executor.cores", CPU_CORES), \
             ("spark.executor.heartbeatInterval", "3600s"), \
             ("spark.network.timeout", "7200s")])
sc = SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .getOrCreate()

import warnings
warnings.filterwarnings("ignore")

RUN_SAMPLE_CODE = True
TRAIN_FINAL_MODEL = False
SEED_NUMBER = 1324

In [ ]:
# Function to print time taken by a particular process, given the start and end times
def printElapsedTime(startTime, endTime):
    elapsedTime = endTime - startTime
    print("-- Process time = %.2f seconds --"%(elapsedTime))

## Method 1: Training models using TFIDF vectorized data

First, we will use the TFIDF vectorized data to build a baseline Naive Bayes model and then train a neural network with 2 hidden layers.

### Loading TFIDF train/ test data

We will begin by loading the train/ test data.


In [ ]:
if RUN_SAMPLE_CODE:
    sourceDir = "data/sample/tfData"
    X_train_tf = sparse.load_npz(sourceDir + "/X_train.npz")
    X_test_tf = sparse.load_npz(sourceDir + "/X_test.npz")

    X_train_tf.sort_indices()
    X_test_tf.sort_indices()

    y_train_tf = pd.read_csv(sourceDir + "/y_train.csv")["review_sentiment"].to_numpy()
    y_test_tf = pd.read_csv(sourceDir + "/y_test.csv")["review_sentiment"].to_numpy()

    print("X_train_tf is of type %s and shape %s."%(type(X_train_tf), X_train_tf.shape))
    print("y_train_tf is of type %s, shape %s and %d unique classes."%(type(y_train_tf), y_train_tf.shape, len(np.unique(y_train_tf))))

### Predictions using a Naive Bayes model for setting a baseline

**ComplementNB** implements the Complement Naive Bayes (CNB) algorithm. CNB is an adaptation of the standard multinomial naive Bayes (MNB) algorithm that is particularly suited for imbalanced data sets. CNB regularly outperforms MNB on text classification tasks so we will be using this model for our baseline.

In [ ]:
if RUN_SAMPLE_CODE:
    tfCNBModel = ComplementNB().fit(X_train_tf, y_train_tf)
    print("ComplementNB Accuracy: %.2f%%"%(100 * metrics.accuracy_score(y_test_tf, tfCNBModel.predict(X_test_tf))))

### Predictions using a Keras Neural Network

We will be using a **Sequential** model with **two** hidden layers and a **sigmoid** activation for the output layer. We can experiment with the hyperparameters such as *L2 regularization, dropout rate, number of nodes in the hidden layers and the activation functions* to find the best possible combination that gives the best accuracy on the test data.

In [ ]:
# Plot the model accuracy and loss over the training epochs
def plotTrainingPerformance(history, figTitle, figSize=(12,5)):
    fig = plt.figure(figsize=figSize)
    sns.set_theme()
    sns.set_style("white")
    
    metrics = history.model.metrics_names
    xvals = np.arange(len(history.history[metrics[0]])) + 1

    for i in range(len(metrics)):
        fig.add_subplot(1, len(metrics), i + 1)
        sns.lineplot(x=xvals, y=history.history[metrics[i]])
        sns.lineplot(x=xvals, y=history.history["val_" + metrics[i]])
        plt.xticks(xvals)
        plt.ylabel(metrics[i])
    
    fig.suptitle(figTitle)
    plt.show()

In [ ]:
# Function to compile, fit and predict keras model
def fitAndPredictModel(modelName, model, X_train, y_train, X_test, y_test, loss, optimizer, metrics, validationSplit, epochs, batch_size):
    # Compile the model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)
    # Split training data into training/ validation sets
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train.reshape(-1,1), test_size=validationSplit, shuffle= True)
    # Fit the model on the training data
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid))
    # Plot training performance
    plotTrainingPerformance(history=history, figTitle="Accuracy/ Loss over Epochs")
    # Predict review sentiments on the test data and check model accuracy
    _, accuracy = model.evaluate(X_test, y_test.reshape((-1,1)))
    print("%s Accuracy: %.2f%%" % (modelName, accuracy*100))

In [ ]:
if RUN_SAMPLE_CODE:

    # Model definition
    tfModel = Sequential()
    l2Reg = 1e-3
    dropout = 0.3
    tfModel.add(Dense(256, input_shape=(X_train_tf.shape[1],), \
        kernel_regularizer=regularizers.l2(l2Reg), \
        bias_regularizer=regularizers.l2(l2Reg)))
    tfModel.add(Activation('relu'))
    tfModel.add(Dropout(dropout))
    tfModel.add(Dense(256, input_shape=(X_train_tf.shape[1],), \
        kernel_regularizer=regularizers.l2(l2Reg), \
        bias_regularizer=regularizers.l2(l2Reg)))
    tfModel.add(Activation('relu'))
    tfModel.add(Dropout(dropout))
    tfModel.add(Dense(1))
    tfModel.add(Activation('sigmoid'))

    print(tfModel.summary())
    

In [ ]:
if RUN_SAMPLE_CODE:
    # Compile, fit and predict model
    fitAndPredictModel(
        modelName="Neural Network", model=tfModel, X_train=X_train_tf, y_train=y_train_tf, X_test=X_test_tf, y_test=y_test_tf, 
        loss="binary_crossentropy", optimizer="adam", metrics=["binary_accuracy"], validationSplit=0.2, epochs=3, batch_size=64)

## Method 2: Training models using sequential word vectors data

Now, we will use the sequential word vectors data to train a recurrent neural network with **1 LSTM layer**, **1 Dense layer** and a **sigmoid** output layer.

### Loading sample sequential train/ test data

We will begin by loading the train/ test data.

In [ ]:
if RUN_SAMPLE_CODE:
    sourceDir = "data/sample/seqData"
    X_train_seq = np.load(sourceDir + "/X_train.npy")
    X_test_seq = np.load(sourceDir + "/X_test.npy")
    y_train_seq = pd.read_csv(sourceDir + "/y_train.csv")["review_sentiment"].to_numpy()
    y_test_seq = pd.read_csv(sourceDir + "/y_test.csv")["review_sentiment"].to_numpy()

    vocabCount = (max([max(x) for x in X_train_seq]) + 1)

### Predictions using a Keras LSTM Recurrent Neural Network

We will be using a **Sequential** model with **one** .... and a **sigmoid** activation for the output layer. We can experiment with the hyperparameters such as *L2 regularization, dropout rate, number of nodes in the hidden layers and the activation functions* to find the best possible combination that gives the best accuracy on the test data.

In [ ]:
if RUN_SAMPLE_CODE:
    # Model definition
    seqModel = Sequential()
    l2Reg = 1e-2
    dropout = 0.5
    seqModel.add(Embedding(input_dim=vocabCount, output_dim=64, input_length=X_train_seq.shape[1], mask_zero=True))
    seqModel.add(LSTM(128, return_sequences=False, dropout=dropout, recurrent_dropout=dropout))
    seqModel.add(Dense(64, activation='relu', \
        kernel_regularizer=regularizers.l2(l2Reg), \
        bias_regularizer=regularizers.l2(l2Reg)))
    seqModel.add(Dropout(dropout))
    seqModel.add(Dense(1, activation='sigmoid'))

    print(seqModel.summary())

In [ ]:
# Compile, fit and predict model
if RUN_SAMPLE_CODE:
    fitAndPredictModel(
        modelName="LSTM RNN", model=seqModel, X_train=X_train_seq, y_train=y_train_seq, X_test=X_test_seq, y_test=y_test_seq, 
        loss="binary_crossentropy", optimizer="rmsprop", metrics=["binary_accuracy"], validationSplit=0.2, epochs=3, batch_size=64)

## Final Model - Keras LSTM Model with Padded Sequential Feature Vectors

In [ ]:
if TRAIN_FINAL_MODEL:
    trainSeq = spark.read.parquet("data/trainSeq.parquet")
    print("There are %d samples in the training data."%(trainSeq.count()))
    trainSeq.show(5)

In [ ]:
if TRAIN_FINAL_MODEL:
    df = trainSeq.toPandas()
    X_train = pd.DataFrame(df["features"].to_list())
    y_train = df["review_sentiment"].to_numpy()
    vocabCount = X_train.max().max()

In [ ]:
if TRAIN_FINAL_MODEL:
    finModel = Sequential()
    l2Reg = 1e-2
    dropout = 0.5
    finModel.add(Embedding(input_dim=vocabCount, output_dim=64, input_length=X_train.shape[1], mask_zero=True))
    finModel.add(LSTM(128, return_sequences=False, dropout=dropout, recurrent_dropout=dropout))
    finModel.add(Dense(64, activation='relu', \
        kernel_regularizer=regularizers.l2(l2Reg), \
        bias_regularizer=regularizers.l2(l2Reg)))
    finModel.add(Dropout(dropout))
    finModel.add(Dense(1, activation='sigmoid'))

    print(finModel.summary())

In [ ]:
if TRAIN_FINAL_MODEL:
        fitAndPredictModel(
                modelName="LSTM RNN", model=finModel, X_train=X_train, y_train=y_train, X_test=X_train, y_test=y_train, 
                loss="binary_crossentropy", optimizer="rmsprop", metrics=["accuracy"], validationSplit=0.2, epochs=3, batch_size=64)
